# Pull Gridmet data to look at correlation
## Variables to compare with physics-based model:
- PDSI  
- Elevation  

From https://climatedataguide.ucar.edu/climate-data/palmer-drought-severity-index-pdsi#:~:text=The%20Palmer%20Drought%20Severity%20Index,more%20extreme%20values%20are%20possible.

The Palmer Drought Severity Index (PDSI) uses readily available temperature and precipitation data to estimate relative dryness. It is a standardized index that generally spans -10 (dry) to +10 (wet). Maps of operational agencies like NOAA typically show a range of -4 to +4, but more extreme values are possible. 

### Environment -- use 'ofp_for_te.yml' 
Base anaconda installation, conda env update with ofp_env_upd.yml (from onhm-fetcher parser Github, added pyviz)


## Setup

In [31]:
# step 0- Import the needed packages
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import geopandas as gpd
import xarray as xr
#import contextily as ctx
import pandas as pd
import os
import numpy as np
from shapely.geometry import Point, polygon
from matplotlib.backends.backend_pdf import PdfPages


Populating the interactive namespace from numpy and matplotlib


C:\Users\galanter\AppData\Local\Continuum\anaconda3\envs\ofp_for_te2\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['var']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [32]:
# step 1- set up directory, plot the TE locations, set up a geopandas dataframe and add columns for the netcdf data
TEdir = r'C:\WBEEP\Thermoelectric-master\Climate_data_fetcher'
TE_df = gpd.read_file(os.path.join(TEdir, '..\GIS','2015_TE_Model_Estimates_lat.long_COMIDs.shp'))
TE_df.reset_index(drop = True, inplace =True)
TE_df = TE_df[TE_df.STATE !='HI']
TE_df = TE_df[TE_df.STATE !='AK']

#get list of plants and change 'NTC/MCRD' name for loop to get data at each point
lst_of_plants = TE_df.loc[:,'PLANT_NAME']
lst_of_plants.update(pd.Series(['NTC_MCRD_Energy_Facility'], index =[737]))


In [33]:
len(lst_of_plants)

1106

In [34]:
# for testing purposes since loop of 1122 plants takes a while! Uncomment these if you want to go test something out

#lst_of_plants = lst_of_plants[0:5]
#lst_of_plants

In [35]:
input_2015 = pd.read_csv(os.path.join(TEdir, '..\TE_Harris_Diehl_2015', '2015_TE_input_data_AEG.csv'))

In [36]:
input_2015.head()

,EIA_PLANT_ID,PLANT_NAME,COUNTY,STATE,NAME_OF_WATER_SOURCE,COOLING_TYPE,MODEL_TYPE,PERCENT_CD_ALLOCATION,ELEVATION_FT,POND_AREA,...,EV_03,EV_04,EV_05,EV_06,EV_07,EV_08,EV_09,EV_10,EV_11,EV_12
0,3,Barry,Mobile,AL,Mobile River,COMPLEX,ONCE-THROUGH RIVER,100,15.0,1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Barry,Mobile,AL,Mobile River,COMPLEX,RECIRCULATING TOWER,100,15.0,1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,Gadsden,Etowah,AL,Coosa River,ONCE-THROUGH FRESH,ONCE-THROUGH RIVER,100,535.0,1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,Gorgas,Walker,AL,Warrior River,ONCE-THROUGH FRESH,ONCE-THROUGH RIVER,100,275.0,1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,Greene County,Greene,AL,Black Warrior River,ONCE-THROUGH FRESH,ONCE-THROUGH RIVER,100,100.0,1000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
conv_ft_mtr = 0.3048
input_2015['ELEVATION_MTR'] = input_2015['ELEVATION_FT']*conv_ft_mtr



In [38]:
elev_phys_mod = input_2015[['EIA_PLANT_ID','ELEVATION_MTR', 'STATE']]
elev_phys_mod = elev_phys_mod.drop_duplicates(subset = 'EIA_PLANT_ID')
# Exclude AK and HI
elev_phys_mod = elev_phys_mod[elev_phys_mod.STATE !='AK']
elev_phys_mod = elev_phys_mod[elev_phys_mod.STATE !='HI']

In [39]:
len(elev_phys_mod)

1106

In [40]:
# step 1- pull elevation from gridMET, units in meters *** or just pull in csv already ran 
#var = 'elev/elev'
dh= 'elevation'
dirPath='http://thredds.northwestknowledge.net:8080'

fileName = '/thredds/dodsC/MET/elev/metdata_elevationdata.nc'
fullfilename= dirPath+fileName
print(fullfilename)
ds = xr.open_dataset(fullfilename)
print(ds, flush =True)

lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['day']
datahandle = ds[dh]
#crshandle=ds['crs']
ts = datahandle.sizes
print(type(ts), flush=True)
print(ts['day'], flush=True)
dayshape = ts['day']
Lonshape = ts['lon']
Latshape = ts['lat']

for i, plant in enumerate(lst_of_plants):
    var = (ds[dh])
    var1 = var.interp(lat =TE_df.iloc[i,5] , lon = TE_df.iloc[i,6])
    var_df = xr.DataArray.to_dataframe(var1)
    var_df = var_df.drop(['lat', 'lon'], axis = 1)   
    
    
    var_df['PLANT_NAME'] = plant 
    var_df['EIA_PLANT_ID'] = TE_df.iloc[i,0]  
    
    if i == 0:
        plants_df = var_df.copy(deep=True)
    else:
        plants_df = plants_df.append(var_df)
        
    print("plant # " + str(i) + " out of 1122 " + "plant name = " + plant)

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/elev/metdata_elevationdata.nc
<xarray.Dataset>
Dimensions:    (day: 1, lat: 585, lon: 1386)
Coordinates:
  * lon        (lon) float32 -124.77216 -124.7305 ... -67.10641 -67.064735
  * lat        (lat) float32 49.396023 49.354355 49.31269 ... 25.104736 25.06307
  * day        (day) datetime64[ns] 1900-01-02
Data variables:
    elevation  (day, lat, lon) float32 ...
Attributes:
    author:   John Abatzoglou - University of Idaho, jabatzoglou@uidaho.edu
    date:     14 November 2011
    note1:    The projection information for this file is: GCS WGS 1984.
    note2:    Citation: Abatzoglou, J.T., 2012, Development of gridded surfac...
<class 'xarray.core.utils.Frozen'>
1
plant # 0 out of 1122 plant name = Barry
plant # 1 out of 1122 plant name = Gadsden
plant # 2 out of 1122 plant name = Gorgas
plant # 3 out of 1122 plant name = Greene County
plant # 4 out of 1122 plant name = E C Gaston
plant # 5 out of 1122 plant name = Browns

plant # 148 out of 1122 plant name = Burlington (IA)
plant # 149 out of 1122 plant name = Ames Electric Services Power Plant
plant # 150 out of 1122 plant name = Streeter Station
plant # 151 out of 1122 plant name = Muscatine Plant #1
plant # 152 out of 1122 plant name = Summit Lake
plant # 153 out of 1122 plant name = Earl F Wisdom
plant # 154 out of 1122 plant name = Cimarron River
plant # 155 out of 1122 plant name = Fort Dodge
plant # 156 out of 1122 plant name = Gordon Evans Energy Center
plant # 157 out of 1122 plant name = La Cygne
plant # 158 out of 1122 plant name = Murray Gill
plant # 159 out of 1122 plant name = Hutchinson Energy Center
plant # 160 out of 1122 plant name = Lawrence Energy Center
plant # 161 out of 1122 plant name = Tecumseh Energy Center
plant # 162 out of 1122 plant name = Coffeyville
plant # 163 out of 1122 plant name = Quindaro
plant # 164 out of 1122 plant name = Pratt
plant # 165 out of 1122 plant name = Wellington 1
plant # 166 out of 1122 plant name =

plant # 308 out of 1122 plant name = Dunkirk Generating Plant
plant # 309 out of 1122 plant name = Nine Mile Point Nuclear Station
plant # 310 out of 1122 plant name = Oswego Harbor Power
plant # 311 out of 1122 plant name = Bowline Point
plant # 312 out of 1122 plant name = S A Carlson
plant # 313 out of 1122 plant name = Asheville
plant # 314 out of 1122 plant name = Roxboro
plant # 315 out of 1122 plant name = G G Allen
plant # 316 out of 1122 plant name = Buck
plant # 317 out of 1122 plant name = James E. Rogers Energy Complex
plant # 318 out of 1122 plant name = Dan River
plant # 319 out of 1122 plant name = Marshall (NC)
plant # 320 out of 1122 plant name = R M Heskett
plant # 321 out of 1122 plant name = Leland Olds
plant # 322 out of 1122 plant name = Milton R Young
plant # 323 out of 1122 plant name = Stanton
plant # 324 out of 1122 plant name = Cardinal
plant # 325 out of 1122 plant name = Miami Fort
plant # 326 out of 1122 plant name = FirstEnergy Ashtabula
plant # 327 out o

plant # 473 out of 1122 plant name = Belle River
plant # 474 out of 1122 plant name = Greenwood (MI)
plant # 475 out of 1122 plant name = Catawba
plant # 476 out of 1122 plant name = McGuire
plant # 477 out of 1122 plant name = Beaver Valley
plant # 478 out of 1122 plant name = H L Spurlock
plant # 479 out of 1122 plant name = Manatee
plant # 480 out of 1122 plant name = Martin
plant # 481 out of 1122 plant name = St Lucie
plant # 482 out of 1122 plant name = Edwin I Hatch
plant # 483 out of 1122 plant name = Wansley
plant # 484 out of 1122 plant name = Big Cajun 2
plant # 485 out of 1122 plant name = R D Morrow
plant # 486 out of 1122 plant name = Nearman Creek
plant # 487 out of 1122 plant name = Iatan
plant # 488 out of 1122 plant name = Jeffrey Energy Center
plant # 489 out of 1122 plant name = Trimble County
plant # 490 out of 1122 plant name = Grand Gulf
plant # 491 out of 1122 plant name = Victor J Daniel Jr
plant # 492 out of 1122 plant name = Colstrip
plant # 493 out of 1122 p

plant # 629 out of 1122 plant name = Arkansas Nuclear One
plant # 630 out of 1122 plant name = Waterford 1 & 2
plant # 631 out of 1122 plant name = Comanche (OK)
plant # 632 out of 1122 plant name = Jim Bridger
plant # 633 out of 1122 plant name = Santan
plant # 634 out of 1122 plant name = Huntington
plant # 635 out of 1122 plant name = Beaver
plant # 636 out of 1122 plant name = General James M Gavin
plant # 637 out of 1122 plant name = Ray D Nixon
plant # 638 out of 1122 plant name = Coyote
plant # 639 out of 1122 plant name = Springerville
plant # 640 out of 1122 plant name = North Valmy
plant # 641 out of 1122 plant name = Cheswick Power Plant
plant # 642 out of 1122 plant name = Astoria Generating Station
plant # 643 out of 1122 plant name = Indian Point 3
plant # 644 out of 1122 plant name = Colorado Energy Nations Company
plant # 645 out of 1122 plant name = Covanta Warren Energy
plant # 646 out of 1122 plant name = Covanta Hennepin Energy
plant # 647 out of 1122 plant name = N

plant # 767 out of 1122 plant name = Covanta Marion Inc
plant # 768 out of 1122 plant name = Covanta Stanislaus Energy
plant # 769 out of 1122 plant name = Covanta Bristol Energy
plant # 770 out of 1122 plant name = ReEnergy Stratton LLC
plant # 771 out of 1122 plant name = Montgomery County Resource Recovery
plant # 772 out of 1122 plant name = Covanta Fairfax Energy
plant # 773 out of 1122 plant name = Pasco Cnty Solid Waste Resource Recovery
plant # 774 out of 1122 plant name = JM Shafer Generating Station
plant # 775 out of 1122 plant name = Pinetree Power Tamworth
plant # 776 out of 1122 plant name = Sterling Power Plant
plant # 777 out of 1122 plant name = Agnews Power Plant
plant # 778 out of 1122 plant name = Viking Energy of McBain
plant # 779 out of 1122 plant name = Viking Energy of Lincoln
plant # 780 out of 1122 plant name = Telogia Power
plant # 781 out of 1122 plant name = Panther Creek Energy Facility
plant # 782 out of 1122 plant name = Parlin Power Plant
plant # 783 o

plant # 904 out of 1122 plant name = Rio Nogales Power Project
plant # 905 out of 1122 plant name = Wolf Hollow I LP
plant # 906 out of 1122 plant name = Hays Energy Project
plant # 907 out of 1122 plant name = Green Country Energy LLC
plant # 908 out of 1122 plant name = La Paloma Generating LLC
plant # 909 out of 1122 plant name = Guadalupe Generating Station
plant # 910 out of 1122 plant name = Lost Pines 1 Power Project
plant # 911 out of 1122 plant name = Bastrop Energy Center
plant # 912 out of 1122 plant name = Granite Ridge
plant # 913 out of 1122 plant name = Bosque County Peaking
plant # 914 out of 1122 plant name = Acadia Energy Center
plant # 915 out of 1122 plant name = Eastman Cogeneration Facility
plant # 916 out of 1122 plant name = South Point Energy Center
plant # 917 out of 1122 plant name = Dogwood Energy Facility
plant # 918 out of 1122 plant name = Rathdrum Power LLC
plant # 919 out of 1122 plant name = Sunrise Power LLC
plant # 920 out of 1122 plant name = Nelson

plant # 1038 out of 1122 plant name = Pinelawn Power LLC
plant # 1039 out of 1122 plant name = TS Power Plant
plant # 1040 out of 1122 plant name = Port Westward
plant # 1041 out of 1122 plant name = Lake Side Power Plant
plant # 1042 out of 1122 plant name = Empire Generating Co  LLC
plant # 1043 out of 1122 plant name = Roseville Energy Park
plant # 1044 out of 1122 plant name = Trigen St. Louis
plant # 1045 out of 1122 plant name = Wygen 2
plant # 1046 out of 1122 plant name = Quail Run Energy Center
plant # 1047 out of 1122 plant name = Colorado Bend Energy Center
plant # 1048 out of 1122 plant name = Clearwater Power Plant
plant # 1049 out of 1122 plant name = Treasure Coast Energy Center
plant # 1050 out of 1122 plant name = West County Energy Center
plant # 1051 out of 1122 plant name = Plum Point Energy Station
plant # 1052 out of 1122 plant name = Russell City Energy Center
plant # 1053 out of 1122 plant name = John W Turk Jr Power Plant
plant # 1054 out of 1122 plant name = J

In [43]:
# elev_df = pd.read_csv(os.path.join(TEdir,'elev_comp.csv'))
TE_df.head()

,EIA_PLANT_,PLANT_NAME,COUNTY,STATE,NAME_OF_WA,LATITUDE,LONGITUDE,COMID,COOLING_TY,GENERATION,...,WATER_TYPE,WITHDRAWAL,CONSUMPTIO,MIN_WITHDR,MAX_WITHDR,MIN_CONSUM,MAX_CONSUM,NET_GENERA,why_no_CID,geometry
0,3,Barry,Mobile,AL,Mobile River,31.006964,-88.010900,18520855.0,COMPLEX,COMPLEX,...,FR,412.0,7.93,189.0,2990.0,6.54,9.13,"11,387,562",None,POINT (-88.01089992 31.00696404)
1,7,Gadsden,Etowah,AL,Coosa River,34.013611,-85.970278,22199835.0,ONCE-THROUGH FRESH,GAS STEAM,...,FR,24.4,0.26,11.1,181.0,0.20,0.32,"214,655",None,POINT (-85.97027778 34.01361111)
2,8,Gorgas,Walker,AL,Warrior River,33.644694,-87.199743,18591860.0,ONCE-THROUGH FRESH,COAL,...,FR,382.0,4.02,174.0,2830.0,3.14,4.91,"4,652,042",None,POINT (-87.19974286 33.64469388)
3,10,Greene County,Greene,AL,Black Warrior River,32.601935,-87.782006,18211194.0,ONCE-THROUGH FRESH,COAL,...,FR,195.0,2.18,89.1,1430.0,1.70,2.66,"2,207,648",None,POINT (-87.78200563 32.60193544)
4,26,E C Gaston,Shelby,AL,Coosa River,33.244058,-86.457463,22273418.0,COMPLEX,COAL,...,FR,142.0,7.56,66.2,997.0,6.51,8.23,"6,578,669",None,POINT (-86.45746253999999 33.24405797)


In [44]:
#merge to dataframes on EIA_PLANT_ID
# not necessary if pulling in csv
elev_df = pd.merge(plants_df, elev_phys_mod, on = 'EIA_PLANT_ID')
elev_df = pd.merge(elev_df, TE_df, left_on ='EIA_PLANT_ID', right_on = 'EIA_PLANT_')

In [45]:
len(elev_df)

1106

In [46]:
elev_df = elev_df[['EIA_PLANT_ID', 'PLANT_NAME_x','LATITUDE', 'LONGITUDE','ELEVATION_MTR', 'elevation']]

In [47]:
#rename columns
cols = ['EIA_PLANT_ID', 'PLANT_NAME','LATITUDE', 'LONGITUDE', 'ELEV_phys_mod', 'ELEV_gm']
elev_df.columns = cols
elev_df['PM-gM'] =  elev_df['ELEV_phys_mod'] - elev_df['ELEV_gm']

#

In [48]:
elev_df.to_csv('elev_df_interp.csv')

In [17]:
#turn it into a pandas geodataframe for plotting
geometry = [Point(xy) for xy in zip(elev_df['LONGITUDE'],elev_df['LATITUDE'])]
geometry[:3]

In [18]:
crs = {'init': 'epsg:4326'}
elev_df_gd = gpd.GeoDataFrame(elev_df, crs = crs, geometry= geometry)

In [19]:
lst_of_large_elev_diffs = elev_df_gd[np.absolute(elev_df_gd['PM-gM'])>100]

In [20]:
lst_of_large_elev_diffs


,EIA_PLANT_ID,PLANT_NAME,LATITUDE,LONGITUDE,ELEV_phys_mod,ELEV_gm,PM-gM,geometry
324,2828,Cardinal,40.252031,-80.648719,204.5208,313.399994,-108.879194,POINT (-80.64871855 40.25203144)
332,2866,FirstEnergy W H Sammis,40.531592,-80.631616,208.1784,328.040009,-119.861609,POINT (-80.63161563 40.53159188)
410,3644,Carbon,39.727095,-110.864505,1865.6808,2090.639893,-224.959093,POINT (-110.8645046 39.72709484)
413,3776,Glen Lyn,37.369889,-80.863361,473.0496,651.000000,-177.950400,POINT (-80.863361 37.36988861)
421,3936,Kanawha River,38.206438,-81.423525,189.5856,349.239990,-159.654390,POINT (-81.42352506 38.20643756)
503,6099,Diablo Canyon,35.210993,-120.855062,0.0000,200.000000,-200.000000,POINT (-120.8550621 35.21099282)
634,8069,Huntington,39.379215,-111.079484,1969.0080,2300.600098,-331.592098,POINT (-111.079484 39.37921497)
644,10003,Colorado Energy Nations Company,39.760556,-105.215000,1717.8528,1878.839966,-160.987166,POINT (-105.215 39.76055556)
684,10495,Rumford Cogeneration,44.551747,-70.541103,138.6840,283.239990,-144.555990,POINT (-70.54110300000001 44.551747)
713,10764,Blue Lake Power LLC,40.877535,-123.994921,25.2984,281.833344,-256.534944,POINT (-123.9949211 40.87753527)


In [25]:
lst_of_plants = lst_of_large_elev_diffs.loc[:,'PLANT_NAME']

In [22]:
# for testing
# lst_of_plants = lst_of_plants[0:2]
# lst_of_large_elev_diffs.iloc[0,2]

In [26]:
lst_of_plants

324                            Cardinal
332              FirstEnergy W H Sammis
410                              Carbon
413                            Glen Lyn
421                       Kanawha River
503                       Diablo Canyon
634                          Huntington
644     Colorado Energy Nations Company
684                Rumford Cogeneration
713                 Blue Lake Power LLC
737         Tamarack Energy Partnership
746              Archbald Power Station
781       Panther Creek Energy Facility
807          Sunnyside Cogen Associates
972               Metcalf Energy Center
1070          Langley Gulch Power Plant
Name: PLANT_NAME, dtype: object

In [27]:
# set up lists for loops- 06/16/2020-- add in precip, specific humidity, surface radiation, wind direction, 
# palmer drought index(*only every 10 days, will explore this more), alfalfa ET, vapor pressure deficit

lst_of_vars = ['rmax/rmax', 'rmin/rmin']

lst_of_dh = ['relative_humidity', 'relative_humidity']

lst_of_vars = ['rmax/rmax', 'rmin/rmin', 'tmmn/tmmn','tmmx/tmmx', 'vs/vs', 'pet/pet', 'pr/pr', 'sph/sph', 'srad/srad',
              'th/th', 'etr/etr', 'vpd/vpd']
lst_of_dh = ['relative_humidity', 'relative_humidity', 'air_temperature', 'air_temperature', 
             'wind_speed', 'potential_evapotranspiration', 'precipitation_amount', 'specific_humidity',
            'surface_downwelling_shortwave_flux_in_air', 'wind_from_direction', 'potential_evapotranspiration', 
            'mean_vapor_pressure_deficit']

dirPath='http://thredds.northwestknowledge.net:8080'

In [50]:
#interp the elevation
#var = 'elev/elev'
dh= 'elevation'
dirPath='http://thredds.northwestknowledge.net:8080'

fileName = '/thredds/dodsC/MET/elev/metdata_elevationdata.nc'
fullfilename= dirPath+fileName
print(fullfilename)
ds = xr.open_dataset(fullfilename)
print(ds, flush =True)

lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['day']
datahandle = ds[dh]
#crshandle=ds['crs']
ts = datahandle.sizes
print(type(ts), flush=True)
print(ts['day'], flush=True)
dayshape = ts['day']
Lonshape = ts['lon']
Latshape = ts['lat']

for i, plant in enumerate(lst_of_plants):
    var = (ds[dh])
    var1 = var.interp(lat =lst_of_large_elev_diffs.iloc[i,2] , lon = lst_of_large_elev_diffs.iloc[i,3])
    var_df = xr.DataArray.to_dataframe(var1)
    var_df = var_df.drop(['lat', 'lon'], axis = 1)   
    
    
    var_df['PLANT_NAME'] = plant 
    var_df['EIA_PLANT_ID'] = lst_of_large_elev_diffs.iloc[i,0]  
    
    if i == 0:
        plants_df = var_df.copy(deep=True)
    else:
        plants_df = plants_df.append(var_df)
        
    print("plant # " + str(i) + " out of 1122 " + "plant name = " + plant)

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/elev/metdata_elevationdata.nc
<xarray.Dataset>
Dimensions:    (day: 1, lat: 585, lon: 1386)
Coordinates:
  * lon        (lon) float32 -124.77216 -124.7305 ... -67.10641 -67.064735
  * lat        (lat) float32 49.396023 49.354355 49.31269 ... 25.104736 25.06307
  * day        (day) datetime64[ns] 1900-01-02
Data variables:
    elevation  (day, lat, lon) float32 ...
Attributes:
    author:   John Abatzoglou - University of Idaho, jabatzoglou@uidaho.edu
    date:     14 November 2011
    note1:    The projection information for this file is: GCS WGS 1984.
    note2:    Citation: Abatzoglou, J.T., 2012, Development of gridded surfac...
<class 'xarray.core.utils.Frozen'>
1
plant # 0 out of 1122 plant name = Cardinal
plant # 1 out of 1122 plant name = FirstEnergy W H Sammis
plant # 2 out of 1122 plant name = Carbon
plant # 3 out of 1122 plant name = Glen Lyn
plant # 4 out of 1122 plant name = Kanawha River
plant # 5 out of 1122 pla

In [51]:
plants_df

,elevation,PLANT_NAME,EIA_PLANT_ID
day,,,
1900-01-02,298.188132,Cardinal,2828
1900-01-02,309.583526,FirstEnergy W H Sammis,2866
1900-01-02,2110.179251,Carbon,3644
1900-01-02,620.444211,Glen Lyn,3776
1900-01-02,337.521371,Kanawha River,3936
1900-01-02,116.803354,Diablo Canyon,6099
1900-01-02,2236.976752,Huntington,8069
1900-01-02,1865.329472,Colorado Energy Nations Company,10003
1900-01-02,246.479587,Rumford Cogeneration,10495


In [28]:
# Step 2- Pull the data from gridMET using two loops-  one for variables and one for plants, this takes some time. 
# output csv with data is TE_plants_w_2015_daily_gridMET.csv if you dont want to run this loop
# Pull data from Northwest Knowledge Network. ds is an xarray

for i, val in enumerate(lst_of_vars):
    fileName = '/thredds/dodsC/MET/' + val + '_2015.nc'
    fullfilename= dirPath+fileName
    print(fullfilename)
    fullfilename= dirPath+fileName
    ds = xr.open_dataset(fullfilename)
    print(ds, flush =True)
    lathandle=ds['lat']
    lonhandle=ds['lon']
    timehandle=ds['day']
    datahandle = ds[lst_of_dh[i]]
    crshandle=ds['crs']
    ts = datahandle.sizes
    print(type(ts), flush=True)
    print(ts['day'], flush=True)
    dayshape = ts['day']
    Lonshape = ts['lon']
    Latshape = ts['lat']
    var = (ds[lst_of_dh[i]])
    
    for j, plant in enumerate(lst_of_plants):
        var1 = var.interp(lat =lst_of_large_elev_diffs.iloc[j,2] , lon = lst_of_large_elev_diffs.iloc[j,3])
        var_df1 = xr.DataArray.to_dataframe(var1)
        var_df1 = var_df1.drop(['lat', 'lon'], axis = 1)
        var_df1['PLANT_NAME'] = plant 
        var_df1['EIA_PLANT_'] = lst_of_large_elev_diffs.iloc[j,0] 
        if j == 0:
            plants_df = var_df1.copy(deep=True)
        else:
            plants_df = plants_df.append(var_df1)
            
    plants_df.reset_index(inplace=True)
    plants_df['EIA_P_DATE'] = plants_df['EIA_PLANT_'].astype(str) + "_" + plants_df['day'].astype(str)
    plants_df.rename({lst_of_dh[i]: val}, axis =1, inplace = True)

    if i == 0:
        final_df = plants_df.copy(deep=True)
    else:
        final_df = final_df.merge(plants_df, left_on = 'EIA_P_DATE', right_on = 'EIA_P_DATE', sort = True)
        
    print("var " + str(i) + " out of 11 variables," + "=" + val)

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/rmax/rmax_2015.nc
<xarray.Dataset>
Dimensions:            (crs: 1, day: 365, lat: 585, lon: 1386)
Coordinates:
  * lat                (lat) float64 49.4 49.36 49.32 ... 25.15 25.11 25.07
  * lon                (lon) float64 -124.8 -124.7 -124.7 ... -67.1 -67.06
  * crs                (crs) float32 3.0
  * day                (day) datetime64[ns] 2015-01-01 2015-01-02 ... 2015-12-31
Data variables:
    relative_humidity  (day, lat, lon) float32 ...
Attributes:
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    geospatial_lon_max:         -67.058333300000015
    geospatial_lon_resolution:  0.041666666666666
    geospatial_lat_resolution:  0.041666666666666
    geospa

KeyboardInterrupt: 

In [56]:
interp_df = final_df[['EIA_P_DATE', 'rmax/rmax', 
        'rmin/rmin', 'tmmn/tmmn','tmmx/tmmx', 'vs/vs', 'pet/pet', 'pr/pr', 
        'sph/sph', 'srad/srad','th/th', 'etr/etr', 'vpd/vpd']]


In [57]:
interp_df

,EIA_P_DATE,rmax/rmax,rmin/rmin,tmmn/tmmn,tmmx/tmmx,vs/vs,pet/pet,pr/pr,sph/sph,srad/srad,th/th,etr/etr,vpd/vpd
0,10003_2010-01-01,72.214398,25.623253,266.523627,281.041603,3.224000,1.500000,0.000000,0.002117,92.780695,239.653333,2.449653,0.441429
1,10003_2010-01-02,80.618506,32.977495,267.819886,280.749650,3.400000,1.408320,0.000000,0.002628,91.232696,251.000000,2.216640,0.380997
2,10003_2010-01-03,94.590345,43.870935,264.882613,276.027621,2.350347,0.800000,0.000000,0.002484,65.533708,313.386666,1.208320,0.211664
3,10003_2010-01-04,72.275784,28.344586,265.793274,278.892271,2.524000,1.200000,0.000000,0.002007,95.596374,266.346667,1.873653,0.369931
4,10003_2010-01-05,57.603679,23.994828,270.663308,282.992949,4.065333,2.024000,0.000000,0.002271,95.015362,245.693333,3.297653,0.562997
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,8069_2010-12-27,93.719412,47.037551,263.246795,274.092824,3.224031,0.799768,0.000000,0.002337,99.552814,298.505796,1.224213,0.177375
5836,8069_2010-12-28,91.982017,40.467009,262.170190,274.649414,2.600878,0.825322,0.000000,0.002110,92.250828,183.000000,1.249884,0.207433
5837,8069_2010-12-29,62.781792,36.669666,263.346801,271.672754,6.401341,1.300000,16.441203,0.001542,79.414174,320.501159,2.049884,0.229977
5838,8069_2010-12-30,95.315702,68.671312,257.546805,263.316979,9.149702,0.399768,0.000000,0.001507,86.812490,305.505796,0.624213,0.044965


In [60]:
# verify WB equation from Stull (2011)

T = 20
RH = 50


a = T*np.arctan(0.151977*np.power((RH + 8.313659),(1/2)))
b = np.arctan(T + RH) - np.arctan(RH - 1.676331)
c = 0.00391838*(np.power(RH,(3/2)))*np.arctan(0.023101*RH) - 4.686035

Tw = a + b + c
Tw

13.699341968988136

In [61]:
interp_df.columns

Index(['EIA_P_DATE', 'rh_mx_prcnt', 'rh_mn_prcnt', 'air_tmn_K', 'air_tmx_K',
       'wnd_spd_m_s', 'ref_et_gr_mm', 'precip_mm', 'sp_h_kg_pr_kg',
       'sur_rad_Wm2', 'wnd_dir_dg_clk_frm_N', 'ref_et_alf_mm', 'vpdef_kPa'],
      dtype='object')

In [62]:
#rename column headers
cols = ['EIA_P_DATE', 'rh_mx_prcnt', 'rh_mn_prcnt', 'air_tmn_K', 'air_tmx_K', 'wnd_spd_m_s',
        'ref_et_gr_mm', 'precip_mm', 'sp_h_kg_pr_kg', 'sur_rad_Wm2','wnd_dir_dg_clk_frm_N',
        'ref_et_alf_mm','vpdef_kPa']
interp_df.columns = cols

In [64]:

#add means and calculated columns
rh = interp_df.loc[:,'rh_mx_prcnt':'rh_mn_prcnt']
interp_df['rh_avg_prcnt'] = rh.mean(axis = 1)

temp = interp_df.loc[:,'air_tmn_K':'air_tmx_K']
temp = temp.mean(axis = 1)
interp_df['air_tmp_avg_C'] = temp -273.15

#et = plants_df.loc[:,'ref_et_gr_mm']
interp_df['open_wtr_et_mm'] = interp_df['ref_et_gr_mm']*1.05

T = interp_df['air_tmp_avg_C']
RH = interp_df['rh_avg_prcnt']

a = T*np.arctan(0.151977*np.power((RH + 8.313659),(1/2)))
b = np.arctan(T + RH) - np.arctan(RH - 1.676331)
c = 0.00391838*(np.power(RH,(3/2)))*np.arctan(0.023101*RH) - 4.686035

Tw = a + b + c

interp_df['wb_tmp_C'] = Tw


#re_order columns
interp_df = interp_df[['EIA_P_DATE', 'rh_avg_prcnt', 'air_tmp_avg_C', 'open_wtr_et_mm', 
                       'wb_tmp_C', 'rh_mx_prcnt', 'rh_mn_prcnt', 'sp_h_kg_pr_kg','air_tmn_K', 'air_tmx_K', 'precip_mm',
                       'wnd_spd_m_s','wnd_dir_dg_clk_frm_N','ref_et_gr_mm','ref_et_alf_mm','sur_rad_Wm2','vpdef_kPa']]

#create a minimalist df with only variables for comparison with physics-based model
interp_df_minimalist = interp_df.drop(columns = ['rh_mx_prcnt', 'rh_mn_prcnt', 'rh_avg_prcnt','air_tmn_K', 'air_tmx_K',
                                                 'sp_h_kg_pr_kg','precip_mm','wnd_dir_dg_clk_frm_N','ref_et_gr_mm',
                                                 'ref_et_alf_mm','sur_rad_Wm2','vpdef_kPa'])

#output as a csv 
interp_df_minimalist.to_csv('interp_TE_plants_w_2010_daily_gridMET.csv')
interp_df.to_csv('interp_TE_plants_w_2010_daily_gridMET_including_processing_variables.csv')


In [ ]:
#combine interp_df and final_df

In [ ]:
with PdfPages('elev_compare.pdf') as pdf:
    fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (8,8))
#    mn = np.min(elev_df_gd['RMSE'])
#    mx = np.max(elev_df_gd['RMSE'])
    ax1.set_title('physMod - gridMET')
    elev_df_gd.plot(ax = ax1, legend = True, column = 'PM-gM', cmap = 'gist_rainbow')
    
    ax2.set_title('hist_gridMET - physMod')
    ax2.hist(elev_df_gd['PM-gM'])
    
    #fig.colorbar(cm.ScalarMappable(cmap = 'gist_rainbow'))
    pdf.savefig()
#    plt.clf()

In [ ]:
with PdfPages('elev_compare_greater_than_mean.pdf') as pdf:
    fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (8,8))
#    mn = np.min(elev_df_gd['RMSE'])
#    mx = np.max(elev_df_gd['RMSE'])
    ax1.set_title('gridMET - physMod')
    a.plot(ax = ax1, legend = True, column = 'DIFF', cmap = 'gist_rainbow')
    
    ax2.set_title('hist_gridMET - physMod')
    ax2.hist(a['DIFF'])
    
    #fig.colorbar(cm.ScalarMappable(cmap = 'gist_rainbow'))
    pdf.savefig()
#    plt.clf()

In [ ]:
# step 1- pull pdsi and elevation from gridMET
var = 'pdsi/pdsi'
dh= 'palmer_drought_severity_index'
dirPath='http://thredds.northwestknowledge.net:8080'

fileName = '/thredds/dodsC/MET/' + var + '_2015.nc'
fullfilename= dirPath+fileName
print(fullfilename)
ds = xr.open_dataset(fullfilename)
print(ds, flush =True)

lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['day']
datahandle = ds[dh]
crshandle=ds['crs']
ts = datahandle.sizes
print(type(ts), flush=True)
print(ts['day'], flush=True)
dayshape = ts['day']
Lonshape = ts['lon']
Latshape = ts['lat']
pdsi = (ds[dh])


In [ ]:
def mean_in_year_month(da):
    month_cnt_2015 = (da.day.dt.year.to_index() - 2015) *12 + da.day.dt.month.to_index()
    
    return da.assign_coords(year_month = month_cnt_2015).groupby('year_month').mean()

In [ ]:
ds_1 = mean_in_year_month(ds)

#ds_1 = mean_in_year_month(ds)


In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
ds

In [ ]:
with PdfPages('pdsi2_pdf') as pdf:
    

In [ ]:
month = ['jan','feb','mar','apr', 'may', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov', 'dec']
#month = 'jan'
for m in range(len(month)):
    if m < 8:
        d_s = '2015-0' + str(m+1) +'-01'
        d_e = '2015-0' + str(m+1) + '-20'
    else: 
        d_s = '2015-' + str(m+1) +'-01'
        d_e = '2015-' + str(m+1) + '-20'        

    pdsi_month = ds.palmer_drought_severity_index.loc[d_s:d_e]
    
    
    with PdfPages('pdsi.pdf') as pdf:
        pdsi_month.mean('day').plot()
        pdf.savefig()
        plt.close()
#jan  = ds.palmer_drought_severity_index.loc['2015-01-01':'2015-01-20']


In [ ]:
val

In [ ]:
feb = ds.palmer_drought_severity_index.loc['2015-02-01':'2015-02-20']

In [ ]:
feb.mean('day').plot()

In [ ]:
#is the elevation of the grid cell much different than the elevation of the TE plant location?



In [ ]:
#get list of plants and change 'NTC/MCRD' name for loop to get data at each point
lst_of_plants = TE_df.loc[:,'PLANT_NAME']
lst_of_plants.update(pd.Series(['NTC_MCRD_Energy_Facility'], index =[737]))

In [ ]:
# for testing purposes since loop of 1122 plants takes a while! Uncomment these if you want to go test something out

lst_of_plants = lst_of_plants[0:5]
lst_of_plants

In [ ]:
# set up lists for loops- 06/16/2020-- add in precip, specific humidity, surface radiation, wind direction, 
# palmer drought index(*only every 10 days, will explore this more), alfalfa ET, vapor pressure deficit

lst_of_vars = ['rmax/rmax', 'rmin/rmin', 'tmmn/tmmn','tmmx/tmmx', 'vs/vs', 'pet/pet', 'pr/pr', 'sph/sph', 'srad/srad',
              'th/th', 'etr/etr', 'vpd/vpd']
lst_of_dh = ['relative_humidity', 'relative_humidity', 'air_temperature', 'air_temperature', 
             'wind_speed', 'potential_evapotranspiration', 'precipitation_amount', 'specific_humidity',
            'surface_downwelling_shortwave_flux_in_air', 'wind_from_direction', 'potential_evapotranspiration', 
            'mean_vapor_pressure_deficit']

dirPath='http://thredds.northwestknowledge.net:8080'



In [ ]:
# Step 2- Pull the data from gridMET using two loops-  one for variables and one for plants, this takes some time. 
# Definitely could be faster/better, can work to make this more efficient, but for now here it is.
# On my computer it takes about 1 hour. 
# output csv with data is TE_plants_w_2015_daily_gridMET.csv if you dont want to run this loop

# Pull data from Northwest Knowledge Network. ds is an xarray

for i, plant in enumerate(lst_of_plants):
    for j, val in enumerate(lst_of_vars):
        fileName = '/thredds/dodsC/MET/' + val + '_2015.nc'
        fullfilename= dirPath+fileName
        print(fullfilename)
        ds = xr.open_dataset(fullfilename)
        print(ds, flush =True)
        lathandle=ds['lat']
        lonhandle=ds['lon']
        timehandle=ds['day']
        datahandle = ds[lst_of_dh[j]]
        crshandle=ds['crs']
        ts = datahandle.sizes
        print(type(ts), flush=True)
        print(ts['day'], flush=True)
        dayshape = ts['day']
        Lonshape = ts['lon']
        Latshape = ts['lat']
        var = (ds[lst_of_dh[j]])
        var1 = var.sel(lat =TE_df.iloc[i,5] , lon = TE_df.iloc[i,6] , method = 'nearest')
        var_df1 = xr.DataArray.to_dataframe(var1)
        var_df1 = var_df1.drop(['lat', 'lon'], axis = 1)   

        if j == 0:
            var_df = var_df1.copy(deep=True)
        else:
            var_df = var_df.merge(var_df1, left_index = True, right_index = True)
    
    
    var_df['PLANT_NAME'] = plant 
    var_df['EIA_PLANT_'] = TE_df.iloc[i,0]  
    
    if i == 0:
        plants_df = var_df.copy(deep=True)
    else:
        plants_df = plants_df.append(var_df)
        
    print("plant # " + str(i) + " out of 1122 " + "plant name = " + plant)

In [ ]:
plants_df.head()


In [ ]:
# pull the pdsi data as a separate step since data is only avaialble every 10 days
lst_of_vars = ['pdsi/pdsi']
lst_of_dh = ['palmer_drought_severity_index']
dirPath='http://thredds.northwestknowledge.net:8080'

fileName = '/thredds/dodsC/MET/' + lst_of_vars + '_2015.nc'
ds = xr.open_dataset(fullfilename)
ds['']

for i, plant in enumerate(lst_of_plants):
    for j, val in enumerate(lst_of_vars):
        fileName = '/thredds/dodsC/MET/' + val + '_2015.nc'
        fullfilename= dirPath+fileName
        print(fullfilename)
        ds = xr.open_dataset(fullfilename)
        print(ds, flush =True)
        lathandle=ds['lat']
        lonhandle=ds['lon']
        timehandle=ds['day']
        datahandle = ds[lst_of_dh[j]]
        crshandle=ds['crs']
        ts = datahandle.sizes
        print(type(ts), flush=True)
        print(ts['day'], flush=True)
        dayshape = ts['day']
        Lonshape = ts['lon']
        Latshape = ts['lat']
        var = (ds[lst_of_dh[j]])
        var1 = var.sel(lat =TE_df.iloc[i,5] , lon = TE_df.iloc[i,6] , method = 'nearest')
        var_df1 = xr.DataArray.to_dataframe(var1)
        var_df1 = var_df1.drop(['lat', 'lon'], axis = 1)   

        if j == 0:
            var_df = var_df1.copy(deep=True)
        else:
            var_df = var_df.merge(var_df1, left_index = True, right_index = True)
    
    
    var_df['PLANT_NAME'] = plant 
    var_df['EIA_PLANT_'] = TE_df.iloc[i,0]  
    
    if i == 0:
        pdsi_df = var_df.copy(deep=True)
    else:
        pdsi_df = pdsi_df.append(var_df)
        
    print("plant # " + str(i) + " out of 1122 " + "plant name = " + plant)

In [ ]:
pdsi_df.head()

In [ ]:
pdsi_df.reset_index(inplace=True)
pdsi_df['EIA_P_DATE'] = pdsi_df['EIA_PLANT_'].astype(str) + "_" + pdsi_df['day'].astype(str)

In [ ]:
#output pdsi data
pdsi_df.to_csv('pdsi_at_TE.csv')

### Data Processing

### Empirical expression for wet-bulb temperature
### Eqn (1) from Stull (2011)

Tw = T * atan[0.151977(RH% + 8.313659)^ (1/2)] + atan(T + RH%) - atan(RH% - 1.676331) + 0.00391838(RH%)^(3/2)* atan(0.023101* RH%) - 4.686035


Where 
Tw  = wet bulb temperature (degc) 
T   = air temperature (degc)
RH% = relative humidity (%)

This equation is valid for relative humidities between 5% and 99% and for air temperatures between -20 deg C and
50 deg C, except for situations having both low humidity and cold temperature. Over the valid range, errors in
wet-bulb temperature range from -1 deg C to + 0.65 deg C, with mean absolute error of less than 0.3 deg C.

### Are there areas where rel hum is < 5 % ? Are there areas that have low humidity and cold temperatures? Not worried about > 50 deg C

In [ ]:
# verify WB equation from Stull (2011)

T = 20
RH = 50


a = T*np.arctan(0.151977*np.power((RH + 8.313659),(1/2)))
b = np.arctan(T + RH) - np.arctan(RH - 1.676331)
c = 0.00391838*(np.power(RH,(3/2)))*np.arctan(0.023101*RH) - 4.686035

Tw = a + b + c
Tw

In [ ]:
#Step 3 - Manipulate the dataframe
# calculate average rh and air temp from max and min, convert avg temp from Kelvins to C
# calculate wetbulb from air temp (Stull 2011)
# calculate ET from ref_et

#add a column for EIA_PLANT_DATE
plants_df.reset_index(inplace=True)
plants_df['EIA_P_DATE'] = plants_df['EIA_PLANT_'].astype(str) + "_" + plants_df['day'].astype(str)

#rename column headers
cols = ['day', 'rh_max_%', 'rh_min_%', 'air_tmn_K', 'air_tmx_K', 'wnd_spd_m_s','ref_et_gr_mm', 'precip_mm', 'sp_h_kg/kg', 
        'sur_rad_Wm2','wnd_dir_dg_clk_frm_N','ref_et_alf_mm','vpdef_kPa','PLANT_NAME', 'EIA_PLANT_', 'EIA_P_DATE']
plants_df.columns = cols

#add means and calculated columns
rh = plants_df.loc[:,'rh_max_%':'rh_min_%']
plants_df['rh_avg_%'] = rh.mean(axis = 1)

temp = plants_df.loc[:,'air_tmn_K':'air_tmx_K']
temp = temp.mean(axis = 1)
plants_df['air_tmp_C_avg'] = temp -273.15

#et = plants_df.loc[:,'ref_et_gr_mm']
plants_df['open_wtr_et_mm'] = plants_df['ref_et_gr_mm']*1.05

T = plants_df['air_tmp_C_avg']
RH = plants_df['rh_avg_%']

a = T*np.arctan(0.151977*np.power((RH + 8.313659),(1/2)))
b = np.arctan(T + RH) - np.arctan(RH - 1.676331)
c = 0.00391838*(np.power(RH,(3/2)))*np.arctan(0.023101*RH) - 4.686035

Tw = a + b + c

plants_df['wb_tmp_C'] = Tw


#re_order columns
plants_df = plants_df[['day', 'PLANT_NAME', 'EIA_PLANT_', 'EIA_P_DATE', 'rh_avg_%', 'air_tmp_C_avg', 'open_wtr_et_mm', 
                       'wb_tmp_C', 'rh_max_%', 'rh_min_%', 'sp_h_kg/kg','air_tmn_K', 'air_tmx_K', 'precip_mm','wnd_spd_m_s',
                       'wnd_dir_dg_clk_frm_N','ref_et_gr_mm','ref_et_alf_mm','sur_rad_Wm2','vpdef_kPa']]

#create a minimalist df with only variables for comparison with physics-based model
plants_df_minimalist = plants_df.drop(columns = ['rh_max_%', 'rh_min_%', 'rh_avg_%','air_tmn_K', 'air_tmx_K','sp_h_kg/kg','precip_mm',
                                                 'wnd_dir_dg_clk_frm_N','ref_et_gr_mm','ref_et_alf_mm','sur_rad_Wm2',
                                                 'vpdef_kPa'])

#output as a csv 
plants_df_minimalist.to_csv('TE_plants_w_2015_daily_gridMET.csv')
plants_df.to_csv('TE_plants_w_2015_daily_gridMET_including_processing_variables.csv')
